In [112]:
import numpy as np
import pandas as pd

In [113]:
data = pd.read_csv('data/train.csv', encoding='Big5')
data = data.replace('NR', '0')
data = np.array(data)

In [114]:
hour = 9
feature_num = 18
day_per_month = 20
per_month_row = feature_num * day_per_month
total_month = int(len(data)/per_month_row) 

In [115]:
month_data = []
for i in range(total_month):
    l = data[i * per_month_row: i * per_month_row  + per_month_row]
    month_data.append(l)  

hour_data = []
for i in range(len(month_data)):
    tmp = []
    for j in range(len(month_data[i])):
        tmp.append(month_data[i][j])
    hour_data.append(tmp)

In [116]:
total = []
for i in range(len(hour_data)):
    df = pd.DataFrame(hour_data[i])
    row, col = df.shape
    tmp = None
    for j in range(day_per_month):
        per_day = df.iloc[j*feature_num:j*feature_num+feature_num,:]
        per_day = per_day.iloc[:,3:]
        
        if tmp is not None:
            tmp = pd.concat([tmp.reset_index(drop=True), per_day.reset_index(drop=True)], axis=1, ignore_index=True)
        else:
            tmp = pd.DataFrame(per_day)   
     
    total.append(tmp)

In [117]:
xss = []
yss = []
for i in range(len(total)):
    df = total[i]
    row, col = df.shape
    for j in range(col-hour-1):
        xs = df.iloc[:,j:j+hour]
        xs = xs.values.ravel()
        ys = float(df.iloc[9,j+hour+1])
        if(ys < 0):
            continue
        xss.append(xs)
        yss.append(float(ys))
        
xss = pd.DataFrame(xss).values.astype(np.float)
# xss = np.column_stack(([1] * len(xss) ,xss))
yss = np.array(yss)

## feature scaling

In [118]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(xss)
xss = scaler.transform(xss)
xss = np.column_stack(([1] * len(xss) ,xss))

In [119]:
from IPython.display import clear_output
max_iter = 10 ** 4
epochs = 10 ** 3
lr = 1
w = np.zeros(feature_num * hour + 1)
w_lr = np.zeros(feature_num * hour+1)

for t in range(epochs):
    w_grad = None
    for m in range(max_iter):
        predict = np.dot(xss,w)
        w_grad = -(2 * np.dot(xss.T,(yss - predict)))
        w_lr = w_lr + w_grad ** 2
        w = w - lr/np.sqrt(w_lr) * w_grad
        
    clear_output()
    print(t)
    print(np.sqrt(np.mean([ x*x for x in (yss-predict)])))

419
8.045451268809176


KeyboardInterrupt: 

In [120]:
test = pd.read_csv('data/test.csv', encoding='Big5', header=None)
test = test.replace('NR', '0')

In [122]:
import pickle
with open('model/0225-scalebest.pkl', 'wb') as f:
    pickle.dump(w, f)

In [125]:
ans = pd.read_csv('data/sampleSubmission.csv', encoding='Big5')
test_feature = 18
total_test = []
row, col = test.shape
test_number = int(row/test_feature)
print(test_number)
for i in range(test_number):
    df = test.iloc[i*test_feature:(i+1)*test_feature, 2:]
    xs = df.values.ravel().astype(np.float)
    xs = (scaler.transform([xs])[0])
    xs = np.concatenate(([1], xs))
    val = np.dot(xs,w)
    val = round(val,2)
    ans.iloc[i,1] = val
ans.to_csv('data/0225-scalebest.csv',index=False)

240


In [169]:
test = pd.DataFrame(xss)
xss_o = test[test.index.isin(index)]
test = pd.DataFrame(yss)
yss_o = test[test.index.isin(index)]

In [127]:
with open('model/0225-scalebest.pkl', 'rb') as f:
    w = pickle.load(f)
        
predict = np.dot(xss,w)
error = predict - yss
print(error)
# # index = []
# for i in range(len(error)):
#     if abs(error[i]) <= 20:
#         index.append(i)
# index = set(index)
# # print(error)
# print(len(yss)-len(index))

[-15.63933124   2.32532141  12.453257   ...   5.15379378  -0.32919253
  -6.75394379]


In [128]:
import matplotlib

ImportError: cannot import name 'get_backend' from 'matplotlib' (/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py)